# **2025 Training Performance Analysis: Session Delivery Overview**

## **Introduction**

This document provides an overview of the training performance across 2025. It aims to analyse the number of sessions delivered during the year to understand the overall demand for each course. The analysis supports data-driven planning for future training programs and helps identify areas with higher participation or emerging needs. This aligns with the organisation’s goal to improve training effectiveness and resource allocation.

## **Objectives**

The key objectives of this documentation are to:

* Determine the total number of sessions delivered per course during 2025.
* Identify trends in session frequency across different training categories.
* Highlight areas of growth or decline to guide future program planning.
* Support decisions on course scheduling, instructor allocation, and resource investment.

Success will be measured by the clarity and accuracy of insights drawn from the data, enabling actionable planning for the next training cycle.

## **Scope**

The analysis covers all training sessions delivered between January and December 2025. It includes course titles, frequency, and participation data where available. The scope is limited to quantitative delivery metrics and does not include qualitative feedback or performance evaluation. All data is sourced from the internal training records and validated for consistency.

## **Methodology**

The analysis follows these key steps:

1. Collect and verify session data from the 2025 training records.
2. Organise courses by category and delivery frequency.
3. Apply filters to identify patterns and variations in demand.
4. Summarise findings in tables and charts for clarity.

This structured approach ensures that the data is complete, accurate, and ready for interpretation.

## **Expected Outcomes**

The expected outcomes of this analysis include:

* A clear summary of session delivery volume for each course.
* Insights into training demand trends across the year.
* Recommendations for strategic adjustments in future training plans.

The results will guide planning for the 2026 training calendar, ensuring efficient use of resources and alignment with learner demand.

In [37]:
import pandas as pd

data = pd.read_excel("/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx")
# Check Data Structure
print("Data Structure:")
print(data.info())

Data Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4150 entries, 0 to 4149
Data columns (total 51 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Organizer                            4150 non-null   object 
 1   Facilitator                          4150 non-null   object 
 2   Co-Facilitator                       9 non-null      object 
 3   Course title                         4150 non-null   object 
 4   Course structure                     4130 non-null   object 
 5   Method                               4150 non-null   object 
 6   Start date                           4150 non-null   object 
 7   End date                             4150 non-null   object 
 8   Course city                          1985 non-null   object 
 9   Course country                       1986 non-null   object 
 10  Title                                448 non-null    object 
 11  First Name    

### **Data Selection**

In this step, we select only nine columns that are relevant to analysing session delivery in 2025. These columns capture essential course and scheduling information while excluding unrelated or unused fields. This focused selection improves clarity and processing efficiency during analysis.

The selected columns are:

1. **Organizer** – The company or unit responsible for managing the course.
2. **Facilitator** – The instructor leading the session.
3. **Course Title** – The official name of the course.
4. **Course Structure** – The type of session (for example, in-house or external).
5. **Method** – The delivery mode (for example, live or online).
6. **Start Date** – The date when the course begins.
7. **End Date** – The date when the course concludes.
8. **Course City** – The city where the training took place.
9. **Course Country** – The country where the session was delivered.

These columns provide all the information required to measure session delivery, frequency, and geographic distribution throughout the year.

In [38]:
# Select only the required columns for analysis
selected_columns = [
    'Organizer',
    'Facilitator',
    'Course title',
    'Course structure',
    'Method',
    'Start date',
    'End date',
    'Course city',
    'Course country'
]

# Create a new dataset containing only these columns
course = data[selected_columns]

# Display the first few rows to confirm the selection
print("Filtered course dataset (columns 1–9):")
display(course.head())

Filtered course dataset (columns 1–9):


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN
4,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN


### Duplicate Check and Participant Count

In this step, we define one session and count how many participant records belong to it. This helps us measure session volume without changing the source data.

#### Session Rule
A session is the unique combination of four columns: **Facilitator**, **Course Title**, **Start Date**, and **End Date**.

#### What We Do
1. Find duplicate rows based on the four columns above.  
2. Add a new column **Participants Count**, which stores the size of each group defined by the four columns.  
3. For any row with no duplicate, set **Participants Count** to 1. This column has no missing values.  
4. Produce three checks for quick validation:  
   - **Total rows**: the number of records in the filtered dataset.  
   - **Total participants count**: the sum of *Participants Count* across unique sessions. This equals the number of rows.  
   - **Total sessions**: the number of unique combinations of the four key columns.  
5. Show a short preview so we can confirm the result.

#### Assumptions and Guardrails
- We do not change the data type of any existing column.  
- We only add the **Participants Count** column.  
- All four grouping columns must be present and readable in the dataset.

In [39]:
# Define the session key
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Count rows per unique session, aligned back to each row
group_sizes = course.groupby(key_cols, dropna=False).size()
course['Participants count'] = course[key_cols].merge(
    group_sizes.rename('Participants count').reset_index(),
    on=key_cols,
    how='left'
)['Participants count']

# Safety check: ensure no missing values in the new column, default to 1 if any appear
course['Participants count'] = course['Participants count'].fillna(1).astype(int)

# Summary numbers
total_rows = len(course)
total_sessions = group_sizes.shape[0]
total_participants = int(group_sizes.sum())  # equals total_rows by construction

# Print summaries
print("Total rows:", total_rows)
print("Total participants count:", total_participants)
print("Total sessions:", total_sessions)

# Preview
print("\nPreview:")
display(course.head())

Total rows: 4150
Total participants count: 4150
Total sessions: 246

Preview:


/tmp/ipykernel_37/2200101167.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course['Participants count'] = course[key_cols].merge(
/tmp/ipykernel_37/2200101167.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course['Participants count'] = course['Participants count'].fillna(1).astype(int)


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5
4,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5


### Remove duplicates, keep one row per session

We will drop duplicate rows using the session keys, Facilitator, Course title, Start date, End date. We keep the first row of each session. The Participants count column already stores the group size, so the kept row holds the correct count. No other column types change.

In [40]:
# Keys that define one session
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Remove duplicates, keep the first row per session
course_dedup = course.drop_duplicates(subset=key_cols, keep='first').copy()

# Quick checks
print("Rows before:", len(course))
print("Rows after :", len(course_dedup))
print("Rows removed:", len(course) - len(course_dedup))

print("\nPreview after dedupe:")
display(course_dedup.head())

Rows before: 4150
Rows after : 246
Rows removed: 3904

Preview after dedupe:


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5
8,TKI,Malek Mohammed Ghazo,C-KPI,Open,Live,2025-12-01 00:00:00,16/01/2025,NaN,NaN,24


In [41]:
# Check if there are still duplicates after the cleaning process
remaining_duplicates = course_dedup[course_dedup.duplicated(subset=['Facilitator', 'Course title', 'Start date', 'End date'], keep=False)]

# Display the results
if len(remaining_duplicates) == 0:
    print("✅ No duplicates remain. The data is clean.")
else:
    print(f"⚠️ There are still {len(remaining_duplicates)} duplicates remaining.")
    display(remaining_duplicates.head())

✅ No duplicates remain. The data is clean.


In [42]:
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(course_dedup)

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,nan,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,nan,nan,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,nan,nan,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,nan,nan,5
8,TKI,Malek Mohammed Ghazo,C-KPI,Open,Live,2025-12-01 00:00:00,16/01/2025,nan,nan,24
32,TKI,Raluca Vintila,C-KPI,Open,Live,13/01/2025,17/01/2025,nan,nan,10
42,TKI,Michael Romero,KPI-E,In-house,Face to face,13/01/2025,13/01/2025,Labuan,Malaysia,26
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,Open,Live,12.01.2025,16.01.2025,nan,nan,12
80,TKI,Radu Cocean,"PM, BSC & KPI-M",In-house,Face to face,13/01/2025,14/01/2025,Riyadh,Saudi Arabia,23
103,TKI,Radu Cocean,"PM, BSC & KPI-M",In-house,Face to face,15/01/2025,16/01/2025,Riyadh,Saudi Arabia,33


In [43]:
# Export cleaned data to Excel file
output_path = '/kaggle/working/Course-Season-2025-clean.xlsx'
course_dedup.to_excel(output_path, index=False)

print(f"✅ Data successfully exported to: {output_path}")

✅ Data successfully exported to: /kaggle/working/Course-Season-2025-clean.xlsx


## Date Standardization Overview

### Selected Date Format

* In the dataset, the new columns **Start Date Std** and **End Date Std** remain as true datetime values.  
* For human readability, any string format can be displayed. For review, the format `MM/DD/YYYY` is used to make scanning easier.  
* For final storage or export, the format `YYYY-MM-DD` is recommended because it sorts correctly and avoids ambiguity.

### Reason for Trying Day-First Before Month-First

* Many records outside the United States use the day-first format, while those within the United States use the month-first format.  
* Trying day-first first captures a large portion of international records.  
* Attempting month-first next recovers United States-style dates that were not parsed in the first pass.  
* Dates in the year-first format (`YYYY-MM-DD`) are handled separately, as they are inherently unambiguous.

### Handling Formats Such as `YYYY/MM/DD HH:MM:SS` (for example, `2025-12-01 00:00:00`)

* This string follows the ISO standard year-first format, which means year 2025, month 12, day 01.  
* If in the source data the middle token actually represents the **day** instead of the month, then the string follows `YYYY-DD-MM`. This reflects a data source error or an alternate local convention.  
* No parser can automatically identify this pattern because both 12 and 01 are valid days or months. A rule must be applied.

**Two safe options to correct such cases:**

1. **Global rule**: if a value begins with a four-digit year, treat it as `YYYY-DD-MM` and swap the last two components before parsing.  
2. **Conditional rule**: apply the swap only for specific providers, regions, or files known to use that structure.

The code provided below supports both approaches through a simple configuration switch.

### Output and Review of Ambiguous Records

* Datetime values remain stored as proper datetime objects in the dataset.  
* A **review table** is generated that displays both the original strings and the parsed dates in `MM/DD/YYYY` format for both start and end dates.  
* The review table also includes an **ambiguity flag**, allowing users to filter and quickly inspect uncertain records.

In [44]:
import numpy as np
import re

# Work on the clean copy we already made
# course_dedup exists from the previous step

# ===== 1) Helpers =====

FORCE_YYYY_DD_MM = True  # set to True if your source writes "YYYY-DD-MM" when it starts with a year

def normalize_separators(s: pd.Series) -> pd.Series:
    """Replace '.', '-' with '/' and trim."""
    return s.astype(str).str.strip().str.replace(r'[-\.]', '/', regex=True)

def looks_year_first(txt: str) -> bool:
    """True if string starts with 4 digits followed by a separator: YYYY/..."""
    return bool(re.match(r'^\s*\d{4}[/\-\.]', str(txt)))

def swap_month_day_when_year_first(s: pd.Series) -> pd.Series:
    """
    If FORCE_YYYY_DD_MM is True, convert 'YYYY/MM/DD' to 'YYYY/DD/MM' by swapping the last two parts.
    Works only on strings that look year first.
    """
    if not FORCE_YYYY_DD_MM:
        return s

    def swapper(x):
        x = str(x).strip()
        if looks_year_first(x):
            parts = re.split(r'[/\-\.]', x)
            if len(parts) >= 3 and parts[1].isdigit() and parts[2].isdigit():
                # swap parts[1] and parts[2]
                parts[1], parts[2] = parts[2], parts[1]
                return '/'.join(parts)
        return x

    return s.map(swapper)

def parse_dates_two_pass(series: pd.Series) -> pd.Series:
    """
    Robust parse:
      1) Normalize separators.
      2) Optionally swap month and day when string starts with year, based on FORCE_YYYY_DD_MM.
      3) Handle Excel serial integers.
      4) Pass 1: day first.
      5) Pass 2: month first where needed.
      6) Combine with priority: serial, day-first, month-first.
    """
    s0 = series.copy()
    s_norm = normalize_separators(s0)
    s_norm = swap_month_day_when_year_first(s_norm)

    # Excel serial numbers to datetime
    serial_mask = s_norm.str.fullmatch(r'\d+')
    serial_vals = pd.to_numeric(s_norm.where(serial_mask), errors='coerce')
    parsed_serial = pd.to_datetime(serial_vals, unit='d', origin='1899-12-30', errors='coerce')

    # First pass: day first
    parsed_dayfirst = pd.to_datetime(s_norm, dayfirst=True, errors='coerce')

    # Second pass: month first where needed
    need_second = parsed_dayfirst.isna()
    parsed_monthfirst = pd.to_datetime(s_norm.where(need_second), dayfirst=False, errors='coerce')

    # Combine
    out = parsed_serial.combine_first(parsed_dayfirst).combine_first(parsed_monthfirst)
    return out

def maybe_ambiguous(text: str) -> bool:
    """Flag strings where both first two tokens are 1..12 after normalization."""
    txt = str(text).strip()
    txt = re.sub(r'[-\.]', '/', txt)
    parts = txt.split('/')
    if len(parts) >= 2:
        try:
            a, b = int(parts[0]), int(parts[1])
            return 1 <= a <= 12 and 1 <= b <= 12
        except ValueError:
            return False
    return False

# ===== 2) Build standardised columns, keep originals unchanged =====
course_dedup['Start date std'] = parse_dates_two_pass(course_dedup['Start date'])
course_dedup['End date std']   = parse_dates_two_pass(course_dedup['End date'])

# Ambiguity flags, only when parsing succeeded
course_dedup['Start date ambiguous'] = course_dedup['Start date'].map(maybe_ambiguous) & course_dedup['Start date std'].notna()
course_dedup['End date ambiguous']   = course_dedup['End date'].map(maybe_ambiguous) & course_dedup['End date std'].notna()

# ===== 3) Review tables =====

# Human readable version for quick review: show as MM/DD/YYYY strings
def fmt_mdY(s):
    return s.dt.strftime('%m/%d/%Y')

review_cols = [
    'Organizer', 'Facilitator', 'Course title',
    'Start date', 'End date',
    'Start date std', 'End date std',
    'Start date ambiguous', 'End date ambiguous'
]

# Full review table with formatted strings for easy reading
review_table = course_dedup[review_cols].copy()
review_table['Start date std (MM/DD/YYYY)'] = fmt_mdY(course_dedup['Start date std'])
review_table['End date std (MM/DD/YYYY)']   = fmt_mdY(course_dedup['End date std'])

# Table of only ambiguous rows, showing both start and end side by side
amb_table = review_table[
    review_table['Start date ambiguous'] | review_table['End date ambiguous']
].copy()

# ===== 4) Summary and previews =====
rows_total = len(course_dedup)
start_ok = course_dedup['Start date std'].notna().sum()
end_ok   = course_dedup['End date std'].notna().sum()
print("Rows:", rows_total)
print("Start date parsed:", start_ok, "| NaT:", rows_total - start_ok, "| Ambiguous flags:", int(course_dedup['Start date ambiguous'].sum()))
print("End date parsed  :", end_ok,   "| NaT:", rows_total - end_ok,   "| Ambiguous flags:", int(course_dedup['End date ambiguous'].sum()))

print("\nPreview, full review table:")
display(review_table.head())

print("\nAmbiguous rows to review, shows both start and end:")
display(amb_table[['Organizer', 'Facilitator', 'Course title',
                   'Start date', 'Start date std (MM/DD/YYYY)',
                   'End date',   'End date std (MM/DD/YYYY)',
                   'Start date ambiguous', 'End date ambiguous']].head(20))

/tmp/ipykernel_37/3823060235.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_monthfirst = pd.to_datetime(s_norm.where(need_second), dayfirst=False, errors='coerce')


Rows: 246
Start date parsed: 246 | NaT: 0 | Ambiguous flags: 37
End date parsed  : 244 | NaT: 2 | Ambiguous flags: 32

Preview, full review table:


,Organizer,Facilitator,Course title,Start date,End date,Start date std,End date std,Start date ambiguous,End date ambiguous,Start date std (MM/DD/YYYY),End date std (MM/DD/YYYY)
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,19/2/2025,23/2/2025,2025-02-19,2025-02-23,False,False,02/19/2025,02/23/2025
1,TKI,Yasser Ghonimy,C-SBP,25/02/2025,29/02/2025,2025-02-25,NaT,False,False,02/25/2025,NaN
2,TKI,Amany Fakhry,C-KPI,19/05/2025,23/05/2025,2025-05-19,2025-05-23,False,False,05/19/2025,05/23/2025
3,TKI,Radu Cocean,C-SBP,06.01.2025,10.01.2025,2025-01-06,2025-01-10,True,True,01/06/2025,01/10/2025
8,TKI,Malek Mohammed Ghazo,C-KPI,2025-12-01 00:00:00,16/01/2025,2025-12-01,2025-01-16,False,False,12/01/2025,01/16/2025



Ambiguous rows to review, shows both start and end:


,Organizer,Facilitator,Course title,Start date,Start date std (MM/DD/YYYY),End date,End date std (MM/DD/YYYY),Start date ambiguous,End date ambiguous
3,TKI,Radu Cocean,C-SBP,06.01.2025,01/06/2025,10.01.2025,01/10/2025,True,True
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,12.01.2025,01/12/2025,16.01.2025,01/16/2025,True,False
419,TKI,Teodora Gorski,C-KPI,3.02.2025,02/03/2025,7.02.2025,02/07/2025,True,True
458,PwC,Abdulaziz Hussein,C-SBP,02.02.2025,02/02/2025,04.02.2025,02/04/2025,True,True
525,TKI,Malek Mohammed Ghazo,C-KPIPP,09.02.2025,02/09/2025,13.02.2025,02/13/2025,True,False
573,TKI,Iulia Tutulan,C-EPM,10.02.2025,02/10/2025,14.02.2025,02/14/2025,True,False
617,TKI,Mariham Magdy,KPI-E,9.02.2025,02/09/2025,9.02.2025,02/09/2025,True,True
633,TKI,Mariham Magdy,KPI-E,10.02.2025,02/10/2025,10.02.2025,02/10/2025,True,True
867,TKI,Dr. Ziyad Jawabra,C-PM,2.3.2025,03/02/2025,6.03.2025,03/06/2025,True,True
943,TKI,Abdulaziz Hussein,C-SBP,09.03.2025,03/09/2025,13.03.2025,03/13/2025,True,False


## Deciding How to Handle the Special Year-First Case

* If all year-first strings such as `2025-12-01 00:00:00` represent **January 12, 2025**, set `FORCE_YYYY_DD_MM = True`. The helper function will then swap the last two parts before parsing.  
* If only certain providers follow that pattern, keep the flag set to `False` and apply a conditional rule instead. For instance, apply the swap only when **Organizer** appears in a specific list or when **Course Country** matches a certain value.  
* Custom rules can be added for these cases, allowing flexible handling of known data sources or regional formats.

## Date Review and Fix Plan

### Scope
* Keep all original date columns unchanged.
* Work only on **Start Date Std** and **End Date Std**.

### Step 1, resolve NaT rows
* List all rows where either standardised column is NaT in a clear review table.
* The reviewer enters corrected date strings in a small dictionary.
* Reparse and update only **Start Date Std** and **End Date Std** based on that dictionary.

### Step 2, resolve Ambiguous rows
* List all rows marked as Ambiguous in a clear review table.
* For each row, the reviewer chooses the rule, day first or month first, or provides an explicit corrected string.
* Reparse and update only the two standardised columns.

### Guarantees
* No other column type is changed.
* Only **Start Date Std** and **End Date Std** are updated.

### 1) Review NaT rows

In [45]:
# Build a table of rows where either standardised date is NaT
nat_mask = course_dedup['Start date std'].isna() | course_dedup['End date std'].isna()
nat_table = course_dedup.loc[nat_mask, [
    'Organizer', 'Facilitator', 'Course title',
    'Start date', 'Start date std',
    'End date',   'End date std'
]].copy()

print(f"NaT rows: {len(nat_table)}")
display(nat_table.head(30))  # show more if you want

NaT rows: 2


,Organizer,Facilitator,Course title,Start date,Start date std,End date,End date std
1,TKI,Yasser Ghonimy,C-SBP,25/02/2025,2025-02-25,29/02/2025,NaT
2338,TKI,Yassine Chaker,C-B,22/6/2025,2025-06-22,26/6/20225,NaT


Note: 29 Feb 2025 is not a valid date, so it will stay NaT. I wil replace it with 01 Mar 2025.

In [46]:
# Apply two manual fixes to End date std on course_dedup
fixes = {
    1:    "01/03/2025",
    2338: "26/06/2025"
}

for idx, txt in fixes.items():
    if idx in course_dedup.index:
        course_dedup.at[idx, 'End date std'] = parse_dates_two_pass(pd.Series([txt])).iat[0]

# Recheck NaT
nat_mask = course_dedup['Start date std'].isna() | course_dedup['End date std'].isna()
print("NaT remaining:", int(nat_mask.sum()))

# Show remaining NaT rows for review
display(course_dedup.loc[nat_mask, [
    'Organizer','Facilitator','Course title',
    'Start date','Start date std',
    'End date','End date std'
]].head(30))

NaT remaining: 0


,Organizer,Facilitator,Course title,Start date,Start date std,End date,End date std


### 2) Review Ambiguous rows

In [47]:
# Recompute ambiguity flags
course_dedup['Start date ambiguous'] = course_dedup['Start date'].map(maybe_ambiguous) & course_dedup['Start date std'].notna()
course_dedup['End date ambiguous']   = course_dedup['End date'].map(maybe_ambiguous) & course_dedup['End date std'].notna()

amb_mask = course_dedup['Start date ambiguous'] | course_dedup['End date ambiguous']
amb_table = course_dedup.loc[amb_mask, [
    'Organizer','Facilitator','Course title',
    'Start date','Start date std',
    'End date','End date std',
    'Start date ambiguous','End date ambiguous'
]].copy()

# Readable strings for quick scanning
amb_table['Start date std (MM/DD/YYYY)'] = course_dedup.loc[amb_mask,'Start date std'].dt.strftime('%m/%d/%Y')
amb_table['End date std (MM/DD/YYYY)']   = course_dedup.loc[amb_mask,'End date std'].dt.strftime('%m/%d/%Y')

print("Ambiguous rows:", len(amb_table))
display(amb_table.head(40))

Ambiguous rows: 40


,Organizer,Facilitator,Course title,Start date,Start date std,End date,End date std,Start date ambiguous,End date ambiguous,Start date std (MM/DD/YYYY),End date std (MM/DD/YYYY)
3,TKI,Radu Cocean,C-SBP,06.01.2025,2025-01-06,10.01.2025,2025-01-10,True,True,01/06/2025,01/10/2025
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,12.01.2025,2025-01-12,16.01.2025,2025-01-16,True,False,01/12/2025,01/16/2025
419,TKI,Teodora Gorski,C-KPI,3.02.2025,2025-02-03,7.02.2025,2025-02-07,True,True,02/03/2025,02/07/2025
458,PwC,Abdulaziz Hussein,C-SBP,02.02.2025,2025-02-02,04.02.2025,2025-02-04,True,True,02/02/2025,02/04/2025
525,TKI,Malek Mohammed Ghazo,C-KPIPP,09.02.2025,2025-02-09,13.02.2025,2025-02-13,True,False,02/09/2025,02/13/2025
573,TKI,Iulia Tutulan,C-EPM,10.02.2025,2025-02-10,14.02.2025,2025-02-14,True,False,02/10/2025,02/14/2025
617,TKI,Mariham Magdy,KPI-E,9.02.2025,2025-02-09,9.02.2025,2025-02-09,True,True,02/09/2025,02/09/2025
633,TKI,Mariham Magdy,KPI-E,10.02.2025,2025-02-10,10.02.2025,2025-02-10,True,True,02/10/2025,02/10/2025
867,TKI,Dr. Ziyad Jawabra,C-PM,2.3.2025,2025-03-02,6.03.2025,2025-03-06,True,True,03/02/2025,03/06/2025
943,TKI,Abdulaziz Hussein,C-SBP,09.03.2025,2025-03-09,13.03.2025,2025-03-13,True,False,03/09/2025,03/13/2025


## Add Full Course Name and Summarise 2025 Sessions

### What will be done
1. Read the grouped sheet from the Excel file at `/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx`.  
2. Map **Course Title** acronyms to full names and add a new column, **Course Name**.  
3. Filter sessions to calendar year **2025**.  
4. Print totals for **sessions** and **participants**.  
5. Build quick summaries by **Course Name**, **Facilitator**, **Month**, **Method**, and **Country**.  
6. Show short head previews for each summary.  
7. Optionally export all outputs to a single Excel file.

### Outputs
* A dataset with an added **Course Name** column.  
* Counts for total sessions and total participants in 2025.  
* Summary tables by course, facilitator, month, method, and country.  
* Head previews for quick checks.  
* One optional Excel export that contains all results.

### Assumptions and guardrails
* No existing column types are changed.  
* Only the **Course Name** column is added.  
* The mapping for acronyms to full names is complete and correct, or any missing items are reported.  
* Year filtering uses the start date of each session.  

In [48]:
import pandas as pd

# ----- 1) Read mapping from the 'grouped' sheet (no merge, use dict) -----
mapping_path = "/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx"
grouped_map = pd.read_excel(mapping_path, sheet_name="grouped")

# Build a clean dict: ACRONYM -> Full name
map_df = (
    grouped_map[['Acronym', 'Product Name']]
    .dropna(subset=['Acronym'])
    .drop_duplicates(subset=['Acronym'])
    .rename(columns={'Product Name': 'Course name'})
)
map_df['acronym_key'] = map_df['Acronym'].astype(str).str.strip().str.upper()
name_dict = dict(zip(map_df['acronym_key'], map_df['Course name']))

# Create a join key on course_dedup
acronym_key = course_dedup['Course title'].astype(str).str.strip().str.upper()

# Safely set or overwrite 'Course name' using the dict, fallback to original Course title
course_dedup['Course name'] = acronym_key.map(name_dict).fillna(course_dedup['Course title'])

# Quick preview of mapping results
print("Preview of Course title -> Course name:")
display(course_dedup[['Course title', 'Course name']].drop_duplicates().head(20))

# ----- 2) Filter to sessions in 2025 (using Start date std) -----
data_2025 = course_dedup[course_dedup['Start date std'].dt.year == 2025].copy()

# ----- 3) Totals -----
total_sessions_2025 = len(data_2025)
total_participants_2025 = int(data_2025['Participants count'].sum())

print("Total sessions 2025:", total_sessions_2025)
print("Total participants 2025:", total_participants_2025)

# ----- 4) Summaries -----
# By course (use full Course name)
by_course = (
    data_2025
    .groupby('Course name', as_index=False)
    .agg(Sessions=('Course name', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By facilitator
by_facilitator = (
    data_2025
    .groupby('Facilitator', as_index=False)
    .agg(Sessions=('Facilitator', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By month
data_2025['Month'] = data_2025['Start date std'].dt.to_period('M').dt.to_timestamp()
by_month = (
    data_2025
    .groupby('Month', as_index=False)
    .agg(Sessions=('Month', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values('Month')
)

# By method
by_method = (
    data_2025
    .groupby('Method', as_index=False)
    .agg(Sessions=('Method', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By country
by_country = (
    data_2025
    .groupby('Course country', as_index=False)
    .agg(Sessions=('Course country', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# ----- 5) Previews -----
print("\nPreview, data_2025:")
display(data_2025.head())

print("\nSessions by course (full name):")
display(by_course.head(20))

print("\nSessions by facilitator:")
display(by_facilitator.head(20))

print("\nSessions by month:")
display(by_month.head(20))

print("\nSessions by method:")
display(by_method.head(20))

print("\nSessions by country:")
display(by_country.head(20))

# ----- 6) Optional export -----
EXPORT_TO_EXCEL = True  # set True to save the workbook
OUTPUT_PATH = "/kaggle/working/Course-Season-2025-summaries.xlsx"

if EXPORT_TO_EXCEL:
    with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as xw:
        data_2025.to_excel(xw, index=False, sheet_name='Detail_2025')
        by_course.to_excel(xw, index=False, sheet_name='By_Course')
        by_facilitator.to_excel(xw, index=False, sheet_name='By_Facilitator')
        by_month.to_excel(xw, index=False, sheet_name='By_Month')
        by_method.to_excel(xw, index=False, sheet_name='By_Method')
        by_country.to_excel(xw, index=False, sheet_name='By_Country')
    print(f"\nSaved to: {OUTPUT_PATH}")

Preview of Course title -> Course name:


,Course title,Course name
0,C-KPI,Certified KPI Professional
1,C-SBP,Certified Strategy and Business Planning Profe...
42,KPI-E,KPI Essentials
80,"PM, BSC & KPI-M","PM, BSC & KPI-M"
136,"PM, BSC, & KPI-Awareness","PM, BSC, & KPI-Awareness"
157,"PM,BSC & KPI Masterclass","PM,BSC & KPI Masterclass"
223,C-PM,Certified Performance Management Professional
242,C-BSC,Certified Balanced Scorecard Management System...
326,C-KPIPP,Certified KPI Professional and Practitioner
406,C-PA,C-PA


Total sessions 2025: 246
Total participants 2025: 4150

Preview, data_2025:


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count,Start date std,End date std,Start date ambiguous,End date ambiguous,Course name,Month
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain,1,2025-02-19,2025-02-23,False,False,Certified KPI Professional,2025-02-01
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN,1,2025-02-25,2025-03-01,False,False,Certified Strategy and Business Planning Profe...,2025-02-01
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN,1,2025-05-19,2025-05-23,False,False,Certified KPI Professional,2025-05-01
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5,2025-01-06,2025-01-10,True,True,Certified Strategy and Business Planning Profe...,2025-01-01
8,TKI,Malek Mohammed Ghazo,C-KPI,Open,Live,2025-12-01 00:00:00,16/01/2025,NaN,NaN,24,2025-12-01,2025-01-16,False,False,Certified KPI Professional,2025-12-01



Sessions by course (full name):


...


Sessions by method:


,Method,Sessions,Participants
1,Live,136,2134
0,Face to face,110,2016



Sessions by country:


,Course country,Sessions,Participants
8,Saudi Arabia,76,1517
10,United Arab Emirates,10,106
6,Malaysia,7,86
11,United Kingdom,4,31
1,Brunei Darussalam,3,55
2,Cambodia,2,121
5,Kuwait,2,14
9,Thailand,1,33
7,Qatar,1,10
4,Jordan,1,7



Saved to: /kaggle/working/Course-Season-2025-summaries.xlsx
